# Rodando em simulador e maquina quantica

In [1]:
from qiskit.circuit.library import PauliTwoDesign, TwoLocal, EfficientSU2, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, NFT
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, SamplingVQE
from qiskit.quantum_info import Pauli

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Sampler

from IPython.display import clear_output

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import pickle
import time

In [2]:
resultados_vqe_4 = None
resultados_qaoa_4 = None

with open('pickles/resultados_vqe_4_diferentes_metodos.pickle', 'rb') as file:
    resultados_vqe_4 = pickle.load(file)

with open('pickles/resultados_qaoa_4_diferentes_metodos.pickle', 'rb') as file:
    resultados_qaoa_4 = pickle.load(file)

## Roda para 4 qubits

In [3]:
num_qubits = 4

In [4]:
# Function to create a ring mixer
def create_ring_mixer(num_qubits):
    ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        ring_mixer *= Pauli(''.join(pauli_list))
    return ring_mixer

# Function to create a parity ring mixer
def create_parity_ring_mixer(num_qubits):
    parity_ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        parity_ring_mixer *= Pauli(''.join(pauli_list))
    pauli_list[-1] = 'X'  # Apply X gate on the last qubit
    parity_ring_mixer *= Pauli(''.join(pauli_list))
    return parity_ring_mixer

# Function to create a full mixer
def create_full_mixer(num_qubits):
    return Pauli('X'*num_qubits)

ring_mixer = create_ring_mixer(num_qubits)
full_mixer = create_full_mixer(num_qubits)
parity_ring_mixer = create_parity_ring_mixer(num_qubits)

otimizadores = {'NFT': NFT(maxiter=100),
                'SLSQP': SLSQP(maxiter=100),
                'COBYLA': COBYLA(maxiter=100),
                'L_BFGS_B': L_BFGS_B(maxiter=100)}

mixers = {'mixer_default': None, 
            'ring_mixer': ring_mixer,
            'full_mixer': full_mixer,
            'parity_ring_mixer': parity_ring_mixer}

ansatzes = {'EfficientSU2': EfficientSU2(num_qubits),
            'RealAmplitudes': RealAmplitudes(num_qubits),
            'PauliTwoDesign': PauliTwoDesign(num_qubits, reps=3),
            'TwoLocal': TwoLocal(num_qubits, "ry", "cz", reps=3, entanglement="full")}

# Valores

In [5]:
copia_resultados_vqe_4 = resultados_vqe_4.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_4.index.unique()):

    df_subset = copia_resultados_vqe_4.loc[
        (copia_resultados_vqe_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_4.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_4_comparacao_valor_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [6]:
copia_resultados_vqe_4 = resultados_vqe_4.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_4.index.unique()):

    df_subset = copia_resultados_vqe_4.loc[
        (copia_resultados_vqe_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_4.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_4_comparacao_valor_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [7]:
copia_resultados_qaoa_4 = resultados_qaoa_4.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_4.index.unique()):

    df_subset = copia_resultados_qaoa_4.loc[
        (copia_resultados_qaoa_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_4.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_4_comparacao_valor_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [8]:

copia_resultados_qaoa_4 = resultados_qaoa_4.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True,)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_4.index.unique()):

    df_subset = copia_resultados_qaoa_4.loc[
        (copia_resultados_qaoa_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_4.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_4_comparacao_valor_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

# Probabilidades

In [9]:
resultados_vqe_4

,,parametros,valor_otimo_encontrado,tempo_elapsado_otimizador,probabilidade_valor_otimo,amostras_otimizacao,parametros_simulador,amostras_otimizacao_simulador,valor_otimo_encontrado_simulador,tempo_elapsado_otimizador_simulador,probabilidade_valor_otimo_simulador,parametros_computador_quantico,amostras_otimizacao_computador_quantico,valor_otimo_encontrado_computador_quantico,tempo_elapsado_otimizador_computador_quantico,probabilidade_valor_otimo_computador_quantico
otimizador,ansatz,,,,,,,,,,,,,,,
COBYLA,EfficientSU2,"[-1.5142979390811262, 2.307387606411415, 1.759...",-1.816578,0.946133,0.932617,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[-1.5357360564375644, 2.436604980065447, 1.828...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-1.847985,288.772233,0.92225,None,None,None,None,None
L_BFGS_B,EfficientSU2,"[0.49936624880039365, 1.9533293316304563, 2.39...",-1.072015,3.667540,0.097656,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[0.49936707338507086, 1.9533297024630716, 2.39...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-1.145833,1884.116073,0.11375,None,None,None,None,None
NFT,EfficientSU2,"[3.118488217948747, -1.4596659418871876, 1.632...",-1.904453,3.149668,0.999023,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[3.0320149348350856, -1.4374697011616084, 1.62...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-1.909518,543.244939,0.99625,None,None,None,None,None
SLSQP,EfficientSU2,"[1.498474392455133, -0.7061383139985018, -1.44...",-1.202363,0.101660,0.007812,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[1.4984744073562941, -0.7061383139985018, -1.4...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-1.219997,98.588065,0.00575,None,None,None,None,None
COBYLA,PauliTwoDesign,"[1.0555042795323522, 1.4064360565941822, 1.877...",-1.664922,0.472108,0.015625,"[SolutionSample(x=array([0., 0., 1., 1.]), fva...","[0.21023283973543536, 2.743906701009386, 1.329...","[SolutionSample(x=array([0., 0., 1., 1.]), fva...",-1.563255,260.891062,0.11575,None,None,None,None,None
L_BFGS_B,PauliTwoDesign,"[3.4648036340805257, -0.0755097161375413, -5.3...",-0.962966,1.495825,0.013672,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[3.281898933090192, -0.4160716308498268, -5.94...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",0.342459,947.447119,0.016,None,None,None,None,None
NFT,PauliTwoDesign,"[-5.892166807257425, -1.6385778915958227, 1.61...",-1.750365,3.828763,0.721680,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[-4.015853382926523, -0.7971771376589064, 1.53...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-1.817294,547.744114,0.845,None,None,None,None,None
SLSQP,PauliTwoDesign,"[-9323229.914351651, 12497636.437782174, 15129...",-0.411333,6.338441,0.832031,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[-44740297.558334984, -19162257.20332406, -348...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.20039,6040.314015,0.682,None,None,None,None,None
COBYLA,RealAmplitudes,"[0.12927802779352743, 2.8589781584164125, 1.98...",-1.822332,0.469526,0.138672,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[0.12790918753817143, 2.8589527756091035, 1.98...","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-1.822943,286.229142,0.153,None,None,None,None,None


In [10]:
copia_resultados_vqe_4 = resultados_vqe_4.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_4.index.unique()]
cores = {'probabilidade_valor_otimo': 'blue', 'probabilidade_valor_otimo_simulador': 'orange', 'probabilidade_valor_otimo_computador_quantico':'cyan'}
nomes = {'probabilidade_valor_otimo': 'simulador local', 'probabilidade_valor_otimo_simulador': 'simulador QASM', 'probabilidade_valor_otimo_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_4.index.unique()):

    df_subset = copia_resultados_vqe_4.loc[
        (copia_resultados_vqe_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_4.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['probabilidade_valor_otimo', 'probabilidade_valor_otimo_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_4_comparacao_probabilidade_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [11]:
copia_resultados_vqe_4 = resultados_vqe_4.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_4.index.unique()]
cores = {'probabilidade_valor_otimo': 'blue', 'probabilidade_valor_otimo_simulador': 'orange', 'probabilidade_valor_otimo_computador_quantico':'cyan'}
nomes = {'probabilidade_valor_otimo': 'simulador local', 'probabilidade_valor_otimo_simulador': 'simulador QASM', 'probabilidade_valor_otimo_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_4.index.unique()):

    df_subset = copia_resultados_vqe_4.loc[
        (copia_resultados_vqe_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_4.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['probabilidade_valor_otimo', 'probabilidade_valor_otimo_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_4_comparacao_probabilidade_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [12]:
copia_resultados_qaoa_4 = resultados_qaoa_4.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_4.index.unique()]
cores = {'probabilidade_valor_otimo': 'blue', 'probabilidade_valor_otimo_simulador': 'orange', 'probabilidade_valor_otimo_computador_quantico':'cyan'}
nomes = {'probabilidade_valor_otimo': 'simulador local', 'probabilidade_valor_otimo_simulador': 'simulador QASM', 'probabilidade_valor_otimo_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_4.index.unique()):

    df_subset = copia_resultados_qaoa_4.loc[
        (copia_resultados_qaoa_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_4.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['probabilidade_valor_otimo', 'probabilidade_valor_otimo_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_4_comparacao_probabilidade_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [13]:

copia_resultados_qaoa_4 = resultados_qaoa_4.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_4.index.unique()]
cores = {'probabilidade_valor_otimo': 'blue', 'probabilidade_valor_otimo_simulador': 'orange', 'probabilidade_valor_otimo_computador_quantico':'cyan'}
nomes = {'probabilidade_valor_otimo': 'simulador local', 'probabilidade_valor_otimo_simulador': 'simulador QASM', 'probabilidade_valor_otimo_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True,)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_4.index.unique()):

    df_subset = copia_resultados_qaoa_4.loc[
        (copia_resultados_qaoa_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_4.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['probabilidade_valor_otimo', 'probabilidade_valor_otimo_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_4_comparacao_probabilidade_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()